In [32]:
# Simple DEM Comparison Analysis
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import rasterio
from rasterio.warp import reproject, Resampling, calculate_default_transform
from rasterio.transform import from_bounds
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Additional analysis imports
from scipy import stats
from sklearn.metrics import mean_squared_error, mean_absolute_error
import xarray as xr
DATA_DIR = Path("data")
RESULTS_DIR = Path("results")
RESULTS_DIR.mkdir(exist_ok=True)

TARGET_CRS = 'EPSG:32632'  # UTM 32N
resolution = 10  # meters
NODATA = -9999
saocom_files = list(DATA_DIR.glob("verona*"))

In [21]:
"""Load SAOCOM point data"""
# Try different separators

df = pd.read_csv(saocom_files[0], sep=',')
# Standardize column names
cols = df.columns.str.upper().str.strip()
mapping = {'LAT': 'lat', 'LON': 'lon', 'HEIGHT': 'height', 'COHER': 'coherence'}

for old, new in mapping.items():
    if old in cols:
        df = df.rename(columns={old: new})

# Clean data
for col in ['lat', 'lon', 'height', 'coherence']:
    df[col] = pd.to_numeric(df[col], errors='coerce')

df = df.dropna(subset=['lat', 'lon', 'height'])
df = df[df['coherence'] >= 0.3]  # Filter by coherence

# Convert to GeoDataFrame and reproject
geometry = [Point(lon, lat) for lon, lat in zip(df['lon'], df['lat'])]
gdf = gpd.GeoDataFrame(df, geometry=geometry, crs='EPSG:4326')
gdf = gdf.to_crs(TARGET_CRS)
gdf['x'] = gdf.geometry.x
gdf['y'] = gdf.geometry.y

In [31]:
"""Convert points to regular grid"""
method = 'cubic'
# Define grid bounds
bounds = gdf.total_bounds
xmin, ymin, xmax, ymax = bounds

# Create grid
x = np.arange(xmin, xmax + resolution, resolution)
y = np.arange(ymax, ymin - resolution, -resolution)  # Flip Y direction
grid_x, grid_y = np.meshgrid(x, y)

# Interpolate
points_xy = np.column_stack([gdf['x'], gdf['y']])
values = gdf['height'].values
grid_points = np.column_stack([grid_x.ravel(), grid_y.ravel()])

grid_values = griddata(points_xy, values, grid_points, method=method, fill_value=NODATA)
saocom_grid = grid_values.reshape(grid_x.shape)

# Save as GeoTIFF
transform = from_bounds(xmin, ymin, xmax, ymax, len(x), len(y))

saocom_raster_path = RESULTS_DIR / f"saocom_dem_{method}.tif"
with rasterio.open(saocom_raster_path, 'w', driver='GTiff', height=saocom_grid.shape[0],
                   width=saocom_grid.shape[1], count=1, dtype='float32',
                   crs=TARGET_CRS, transform=transform, nodata=NODATA) as dst:
    dst.write(saocom_grid.astype('float32'), 1)

